In [1]:
import ArcFace
from mtcnn.mtcnn import MTCNN
import matplotlib.pyplot as plt
import cv2
from tensorflow.keras.preprocessing import image
import numpy as np
%matplotlib inline

In [2]:
model = ArcFace.loadModel()
face_detector = MTCNN()

arcface_weights.h5  will be downloaded to  C:\Users\Arianpc/.deepface/weights/arcface_weights.h5


Downloading...
From: https://drive.google.com/uc?id=1LVB3CdVejpmGHM28BpqqkbZP5hDEcdZY
To: C:\Users\Arianpc\.deepface\weights\arcface_weights.h5
100%|██████████| 137M/137M [02:30<00:00, 913kB/s]  


In [3]:
def detect_face(img):
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) #mtcnn expects RGB but OpenCV read BGR
    detections = face_detector.detect_faces(img_rgb)
    detection = detections[0]
    x, y, w, h = detection["box"]
    detected_face = img[int(y):int(y+h), int(x):int(x+w)]
    return detected_face

In [4]:
def preprocess_face(img, target_size=(112,112)):
    img = cv2.imread(img)
    img = detect_face(img)
    img = cv2.resize(img, target_size)
    img_pixels = image.img_to_array(img)
    img_pixels = np.expand_dims(img_pixels, axis = 0)
    img_pixels /= 255 #normalize input in [0, 1]
    return img_pixels

In [5]:
def img_to_encoding(path):
    img = preprocess_face(path)
    return model.predict(img)[0]

In [10]:
database = {}

database["maryam"] = img_to_encoding("maryam1.jpg")
database["mahmoud"] = img_to_encoding("mahmoud1 (1).jpg")
database["masoud"] = img_to_encoding("masoud_fekri1.jpg")

In [11]:
def EuclideanDistance(source_representation, test_representation):
    euclidean_distance = source_representation - test_representation
    euclidean_distance = np.sum(np.multiply(euclidean_distance, euclidean_distance))
    euclidean_distance = np.sqrt(euclidean_distance)
    return euclidean_distance

In [17]:
verification_threshhold = 4.4

In [18]:
def verify(image_path, identity, database):
    # Step 1: Compute the encoding for the image. Use img_to_encoding()
    encoding = img_to_encoding(image_path) 
    
    # Step 2: Compute distance with identity's image
    dist = EuclideanDistance(encoding, database[identity])
    
    # Step 3: Open the door if dist < verification_threshhold, else don't open
    if dist < verification_threshhold:
        print("It's " + str(identity) + ", welcome!")
    else:
        print("It's not " + str(identity) + ", please go away")
             
    return dist

In [22]:


verify("mahmoud2 .jpg", "mahmoud", database)



It's mahmoud, welcome!


5.4052253